## download data

In [2]:
base_url = "https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02"
# save_dir = "/home/geethen/invasives/hypinvalimap/data"
save_dir = "/mnt/hdd1/invasives/hypinvalimap/data"

files_to_download = []

for x in range(32, 36):
    for y in range(16, 22):
        filename = f"AVIRIS-NG_BIOSCAPE_V02_L3_{x}_{y}_RFL.nc"
        url = f"{base_url}/{x}_{y}/{filename}"
        output_path = f"{save_dir}/{filename}"
        files_to_download.append((url, output_path))

# Print to verify
for url, path in files_to_download:
    print(f"('{url}', '{path}'),")


('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02/32_16/AVIRIS-NG_BIOSCAPE_V02_L3_32_16_RFL.nc', '/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_16_RFL.nc'),
('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02/32_17/AVIRIS-NG_BIOSCAPE_V02_L3_32_17_RFL.nc', '/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_17_RFL.nc'),
('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02/32_18/AVIRIS-NG_BIOSCAPE_V02_L3_32_18_RFL.nc', '/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_18_RFL.nc'),
('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02/32_19/AVIRIS-NG_BIOSCAPE_V02_L3_32_19_RFL.nc', '/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_19_RFL.nc'),
('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02/32_20/AVIRIS-NG_BIOSCAPE_V02_L3_32_20_RFL.nc', '/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_20_RFL.nc'),
('https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape

In [3]:
import os
import requests
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from time import sleep

MAX_RETRIES = 3
PARALLEL_DOWNLOADS = 2

def download_file(url, output_path):
    if os.path.exists(output_path):
        return f"⏭️ Skipped (already exists): {output_path}"

    os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)
    for attempt in range(MAX_RETRIES):
        try:
            with requests.get(url, stream=True, timeout=30) as r:
                r.raise_for_status()
                total = int(r.headers.get('content-length', 0))
                with open(output_path, 'wb') as f, tqdm(
                    total=total, unit='B', unit_scale=True, desc=os.path.basename(output_path), ncols=80
                ) as pbar:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
            return f"✅ Downloaded: {output_path}"
        except Exception as e:
            print(f"⚠️ Retry {attempt+1} failed for {url}: {e}")
            sleep(2)
    return f"❌ Failed to download: {url}"

# Parallel download with progress bars
with ThreadPoolExecutor(max_workers=PARALLEL_DOWNLOADS) as executor:
    futures = [executor.submit(download_file, url, path) for url, path in files_to_download]
    for future in futures:
        print(future.result())


⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_16_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_17_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_18_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_19_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_20_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_21_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_33_16_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_33_17_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_33_18_RFL.nc
⏭️ Skipped (already exists): /mnt/hdd1/invasiv

# ML pipeline for data classification

In [1]:
import geopandas as gpd
import xarray as xr
from shapely.geometry import box, mapping
import rioxarray as riox
import numpy as np
import hvplot.xarray
import holoviews as hv
import xvec
import matplotlib.pyplot as plt
import pandas as pd
from dask.diagnostics import ProgressBar
import warnings

warnings.filterwarnings('ignore')
hvplot.extension('bokeh')

In [2]:
ds = xr.open_dataset(r"/mnt/hdd1/invasives/hypinvalimap/data/AVIRIS-NG_BIOSCAPE_V02_L3_32_16_RFL.nc", engine="rasterio", chunks="auto")
ds


<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.4e+05 8.4e+05 ... 8.5e+05 8.5e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
Data variables: (12/13)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sensor_zenith     (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    to_sun_azimuth       (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:05Z
    grid_cell:                         3216
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [14]:
print(ds.rio.crs)

PROJCS["unnamed",GEOGCS["Ellipse Based",DATUM["Ellipse Based",SPHEROID["Unnamed",6378137,298.257223562997]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",-30],PARAMETER["longitude_of_center",25],PARAMETER["standard_parallel_1",-22],PARAMETER["standard_parallel_2",-38],PARAMETER["false_easting",1400000],PARAMETER["false_northing",1300000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [ ]:
import xarray as xr
import geopandas as gpd
from shapely.geometry import box

def get_bounds_as_gdf(ds):
    """
    Given a NetCDF file with georeferenced raster data, returns a GeoDataFrame
    with a single bounding box geometry representing the dataset extent.
    
    Parameters:
        nc_path (str): Path to the NetCDF file.
    
    Returns:
        gpd.GeoDataFrame: A GeoDataFrame with one row containing the bounding box.
    """
    # Extract coordinate variables (commonly named 'x' and 'y')
    try:
        x = ds['x'].values
        y = ds['y'].values
    except KeyError:
        raise ValueError("Could not find coordinate variables 'x' and 'y' in the dataset.")

    # Compute bounds
    bounds = (x.min(), y.min(), x.max(), y.max())
    geom = box(*bounds)

    # Get CRS from dataset
    try:
        crs = ds.rio.crs
    except AttributeError:
        crs = None  # Will be set to None if CRS is missing

    return gpd.GeoDataFrame(index=[0], geometry=[geom], crs=crs).to_crs(epsg=4326)

geo = get_bounds_as_gdf(ds)

# Clip the raw data to the bounding box
# points = joined.clip(geo)
# m = points.explore()
# m = geo.explore(m=m, color="red", name="Bounding Box")
# m


In [9]:
# Divide reflectance values by 1000 before visualization
ds_scaled = ds / 1000
h = ds_scaled.sel(wavelength=[660, 570, 480], method="nearest").hvplot.rgb(
    'x', 'y',
    rasterize=True,
    data_aspect=1,
    bands='wavelength',
    frame_width=400
)
h

BokehModel(combine_events=True, render_bundle={'docs_json': {'dde93648-5a60-4d15-a88a-43a1dde62301': {'version…

In [10]:
# Load invasive species point data
# gdf = gpd.read_file(r'C:\Users\coach\myfiles\postdoc\Invasives\code\hypinvalimap\data\aliens_sep2018.shp')
gdf = gpd.read_file(r'/home/geethen/invasives/hypinvalimap/data/2018_2023_MgnChg.shp')
gdf = gdf[gdf.geometry.notnull()]

gdf2 = gpd.read_file(r'/home/geethen/invasives/hypinvalimap/data/aliens_sep2018.shp')
gdf2.dropna(subset=['geometry'], inplace=True)
gdf2 = gdf2[gdf2.geometry != 'None']
gdf2.shape


# gdf2
# gdf.head()

(1688, 6)

In [11]:
gdf2.isna().sum()

fid          0
class        0
group        0
layer       74
path        74
geometry     0
dtype: int64

In [12]:
joined = gpd.sjoin_nearest(gdf2[['fid', 'class', 'group', 'geometry']], gdf[['geometry','change', 'notes']], how='inner', distance_col='dist')

joined.isna().sum()

fid               0
class             0
group             0
geometry          0
index_right       0
change          577
notes          1450
dist              0
dtype: int64

In [23]:
joined.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [21]:
m = joined.explore()
geo = get_bounds_as_gdf(ds)
m = geo.explore(m=m, color="red", name="Bounding Box")
m

In [27]:
def extract_points(ds, points):
    """
    Extracts data values at specified points from a locally stored dataset.

    Parameters:
    - ds: str, path to the dataset (e.g., netCDF or GeoTIFF) or dataset.
    - points: GeoDataFrame, point locations to extract data.

    Returns:
    - DataFrame containing extracted data values and point indices.
    """

    if isinstance(ds, str):
        ds = xr.open_dataset(ds, engine="rasterio", chunks="auto")
    
    ds_crs = ds.rio.crs
    
    # get the bounding box of the dataset
    geo = get_bounds_as_gdf(ds).to_crs(ds_crs)

    # Reproject points to match the CRS of the dataset
    if points.crs != ds_crs:
        points = points.to_crs(ds_crs)

    # Clip the raw data to the bounding box
    points = points.clip(geo)
    print(f'got {points.shape[0]} point from {ds.title}')

    # Extract data at points
    extracted = ds.xvec.extract_points(
        points['geometry'], 
        x_coords="x", 
        y_coords="y", 
        index=True
    )
    
    return extracted

df = extract_points(ds, joined)
df

got 6 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_16)


<xarray.Dataset> Size: 14kB
Dimensions:              (wavelength: 425, geometry: 6, band: 1)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * geometry             (geometry) object 48B POINT (848040.2261658383 86194...
    index                (geometry) int64 48B 165 1178 1179 1177 1181 1180
Data variables: (12/13)
    reflectance          (wavelength, geometry) float32 10kB dask.array<chunksize=(8, 6), meta=np.ndarray>
    cosine_i             (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    utc_time             (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    earth_sun_distance   (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    scene_id             (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    path_length          (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    ...                   ...
    to_sensor_zenith     (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    to_sun_azimuth       (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    to_sun_zenith        (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    solar_phase          (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    slope                (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
    aspect               (band, geometry) float32 24B dask.array<chunksize=(1, 6), meta=np.ndarray>
Indexes:
    geometry  GeometryIndex (crs=PROJCS["unnamed",GEOGCS["Ellipse Based",DATUM["Ellipse Based",SPHE ...)
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:05Z
    grid_cell:                         3216
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [28]:
base_url = "https://popo.jpl.nasa.gov/pub/PBrodrick/bioscape/mosaic_v02"
# save_dir = "/home/geethen/invasives/hypinvalimap/data"
save_dir = "/mnt/hdd1/invasives/hypinvalimap/data"

files_to_download = []

for x in range(32, 36):
    for y in range(16, 22):
        filename = f"AVIRIS-NG_BIOSCAPE_V02_L3_{x}_{y}_RFL.nc"
        url = f"{base_url}/{x}_{y}/{filename}"
        output_path = f"{save_dir}/{filename}"
        files_to_download.append((url, output_path))
        
df_all = [extract_points(file, joined) for _, file in files_to_download]

#combine the results into a single xarray
df_all  = xr.concat(df_all, dim='file')
df_all

got 6 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_16)
got 174 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_17)
got 248 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_18)
got 151 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_19)
got 34 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_20)
got 14 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 32_21)
got 11 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_16)
got 83 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_17)
got 168 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_18)
got 94 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_19)
got 63 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_20)
got 61 point from AVIRIS-NG L3 Mosaiced Surface Reflectance (grid cell: 33_21)
got 9 point from AVIRIS-NG L3 Mosaiced Surface Re

<xarray.Dataset> Size: 71MB
Dimensions:              (file: 24, wavelength: 425, geometry: 1683, band: 1)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * band                 (band) int64 8B 1
  * geometry             (geometry) object 13kB POINT (848040.2261658383 8619...
    transverse_mercator  int64 8B 0
    index                (file, geometry) float64 323kB 165.0 1.178e+03 ... nan
Dimensions without coordinates: file
Data variables: (12/13)
    reflectance          (file, wavelength, geometry) float32 69MB dask.array<chunksize=(1, 8, 2), meta=np.ndarray>
    cosine_i             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    utc_time             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    earth_sun_distance   (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    scene_id             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    path_length          (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    ...                   ...
    to_sensor_zenith     (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    to_sun_azimuth       (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    to_sun_zenith        (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    solar_phase          (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    slope                (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    aspect               (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
Indexes:
    geometry  GeometryIndex (crs=PROJCS["unnamed",GEOGCS["Ellipse Based",DATUM["Ellipse Based",SPHE ...)
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:05Z
    grid_cell:                         3216
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [30]:
class_xr =joined[['class','group']].to_xarray()
ds = df_all.merge(class_xr.astype(int),join='left')
ds

<xarray.Dataset> Size: 71MB
Dimensions:              (file: 24, wavelength: 425, geometry: 1683, band: 1,
                          index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * band                 (band) int64 8B 1
  * geometry             (geometry) object 13kB POINT (848040.2261658383 8619...
    transverse_mercator  int64 8B 0
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Dimensions without coordinates: file
Data variables: (12/15)
    reflectance          (file, wavelength, geometry) float32 69MB dask.array<chunksize=(1, 8, 2), meta=np.ndarray>
    cosine_i             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    utc_time             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    earth_sun_distance   (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    scene_id             (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    path_length          (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    solar_phase          (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    slope                (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    aspect               (file, band, geometry) float32 162kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Indexes:
    geometry  GeometryIndex (crs=PROJCS["unnamed",GEOGCS["Ellipse Based",DATUM["Ellipse Based",SPHE ...)
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:05Z
    grid_cell:                         3216
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [ ]:
with ProgressBar():
    edf = ds.persist()

[                                        ] | 0% Completed | 83m 2ssss

In [32]:
widedf = ds.head().xvec.to_geodataframe(long=True).pivot_table(
    index=['geometry'],  # Replace with your actual spatial and other relevant dimensions
    columns='wavelength',
    values='reflectance'
).reset_index()
widedf

wavelength,geometry,377.19565,382.20566,387.21564,392.22565,397.22565
0,POINT (848040.226 861940.128),-0.001360,0.001746,0.009233,0.011261,0.012637
1,POINT (848214.467 863279.653),-0.013990,-0.004646,-0.002463,0.006976,0.009752
2,POINT (848339.869 863200.037),-0.004312,-0.003247,0.007766,0.007847,0.007336
3,POINT (848402.595 863181.508),-0.009425,-0.003676,0.004743,0.007010,0.006713
4,POINT (849256.115 863324.094),-0.003740,0.003531,0.010820,0.011867,0.013556


In [ ]:
dsp_plot = ds.where(ds['class']==5, drop=True)
h = dsp_plot['reflectance'].hvplot.line(x='wavelength',by='index',
                                    color='green',ylim=(0,0.5),alpha=0.5,legend=False)
h

In [15]:
missing_cols = gdf.columns[gdf.isnull().any()]
print(missing_cols)


Index(['2018_2023', 'change', 'notes'], dtype='object')


In [ ]:
import geopandas as gpd

# Convert geometry to WKB for compact, lossless binary storage
widedf['geometry'] = widedf['geometry'].apply(lambda geom: geom.wkb)

# Save to Feather
widedf.to_feather("/home/geethen/invasives/hypinvalimap/data/2023_extracted_data.feather")

In [59]:
import pandas as pd 
# Read the Feather file
xdf = pd.read_feather("/home/geethen/invasives/hypinvalimap/data/2023_extracted_data.feather")

# Convert geometry back from WKB
from shapely import wkb
xdf['geometry'] = xdf['geometry'].apply(wkb.loads)

# Restore as a GeoDataFrame with correct CRS if needed
xdf = gpd.GeoDataFrame(xdf, geometry='geometry', crs="EPSG:4326")
xdf

wavelength,geometry,377.19565,382.20566,387.21564,392.22565,397.22565,402.23566,407.24564,412.25565,417.26566,...,2455.7957,2460.8057,2465.8157,2470.8157,2475.8257,2480.8357,2485.8457,2490.8557,2495.8657,2500.8757
0,POINT (19.00879 -33.9911),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
1,POINT (19.00823 -33.99113),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
2,POINT (19.01091 -33.9917),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
3,POINT (19.0071 -33.99783),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
4,POINT (19.0014 -33.99701),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683,POINT (19.50658 -33.98428),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
1684,POINT (19.50876 -33.98645),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
1685,POINT (19.50898 -33.98638),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827
1686,POINT (19.50693 -33.98646),0.014935,0.021665,0.027689,0.031128,0.031805,0.03197,0.035033,0.03688,0.036342,...,0.13415,0.142706,0.139535,0.15612,0.148241,0.171788,0.174089,0.180323,0.165728,0.190827


In [36]:
# Now that df is extracted, release ds
del ds
import gc
gc.collect()

2890

In [ ]:
df.xvec.to_geodataframe().head()import matplotlib.pyplot as plt
import numpy as np

# Assuming prediction_probabilities is your array of shape (844, 11)
# If it's not already a numpy array, you might need to convert it:
# prediction_probabilities = np.array(prediction_probabilities)

n_samples, n_classes = prediction_probabilities.shape

# Determine grid size for subplots (e.g., 4 rows, 3 columns for 11 classes)
n_cols = 3
n_rows = (n_classes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

for i in range(n_classes):
    ax = axes[i]
    ax.hist(prediction_probabilities[:, i], bins=20, edgecolor='black')
    ax.set_title(f'Class {i+1}')
    ax.set_xlabel('Probability')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 1) # Probabilities are between 0 and 1

# Hide any unused subplots
for j in range(n_classes, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Distribution of Class Probabilities', y=1.02, fontsize=16) # Add a main title
plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()
import matplotlib.pyplot as plt
import numpy as np

# Assuming prediction_probabilities is your array of shape (844, 11)
# If it's not already a numpy array, you might need to convert it:
# prediction_probabilities = np.array(prediction_probabilities)

n_samples, n_classes = prediction_probabilities.shape

# Determine grid size for subplots (e.g., 4 rows, 3 columns for 11 classes)
n_cols = 3
n_rows = (n_classes + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))
axes = axes.flatten() # Flatten the 2D array of axes for easy iteration

for i in range(n_classes):
    ax = axes[i]
    ax.hist(prediction_probabilities[:, i], bins=20, edgecolor='black')
    ax.set_title(f'Class {i+1}')
    ax.set_xlabel('Probability')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0, 1) # Probabilities are between 0 and 1

# Hide any unused subplots
for j in range(n_classes, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Distribution of Class Probabilities', y=1.02, fontsize=16) # Add a main title
plt.tight_layout() # Adjust layout to prevent overlapping titles/labels
plt.show()


MemoryError: Unable to allocate 4.51 GiB for an array with shape (425, 1688, 1, 1688) and data type float32

In [11]:
# Add class and group columns to spectral signatures
class_xr = gdf[['class','group']].to_xarray()
ds = ds.merge(class_xr.astype(int),join='left')
ds

<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1,
                          index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [26]:
ds.reflectance.data.shape

(425, 2000, 2000)

In [12]:
with ProgressBar():
    dsp = ds.persist()

[########################################] | 100% Completed | 338.30 s


In [23]:
dsp.isnull().sum(dim='wavelength').compute()  # Check for NaN values in the dataset

<xarray.Dataset> Size: 80MB
Dimensions:              (y: 2000, x: 2000, band: 1, index: 1688)
Coordinates:
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B 0
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (y, x) int64 32MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
    cosine_i             (band, y, x) bool 4MB False False False ... False False
    utc_time             (band, y, x) bool 4MB False False False ... False False
    earth_sun_distance   (band, y, x) bool 4MB False False False ... False False
    scene_id             (band, y, x) bool 4MB False False False ... False False
    path_length          (band, y, x) bool 4MB False False False ... False False
    ...                   ...
    to_sun_zenith        (band, y, x) bool 4MB False False False ... False False
    solar_phase          (band, y, x) bool 4MB False False False ... False False
    slope                (band, y, x) bool 4MB False False False ... False False
    aspect               (band, y, x) bool 4MB False False False ... False False
    class                (index) bool 2kB False False False ... False False
    group                (index) bool 2kB False False False ... False False

In [21]:
dsp.dropna(dim='index', subset=['reflectance'])

<xarray.Dataset> Size: 7GB
Dimensions:              (wavelength: 425, x: 2000, y: 2000, band: 1,
                          index: 1688)
Coordinates:
  * wavelength           (wavelength) float64 3kB 377.2 382.2 ... 2.501e+03
  * x                    (x) float64 16kB 8.5e+05 8.5e+05 ... 8.6e+05 8.6e+05
  * y                    (y) float64 16kB 8.7e+05 8.7e+05 ... 8.6e+05 8.6e+05
    transverse_mercator  int64 8B ...
  * band                 (band) int64 8B 1
  * index                (index) int64 14kB 0 1 2 3 4 ... 1684 1685 1686 1687
Data variables: (12/15)
    reflectance          (wavelength, y, x) float32 7GB dask.array<chunksize=(8, 2000, 2000), meta=np.ndarray>
    cosine_i             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    utc_time             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    earth_sun_distance   (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    scene_id             (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    path_length          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    ...                   ...
    to_sun_zenith        (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    solar_phase          (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    slope                (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    aspect               (band, y, x) float32 16MB dask.array<chunksize=(1, 2000, 2000), meta=np.ndarray>
    class                (index) int64 14kB 0 0 0 0 0 0 0 ... 10 10 10 10 10 10
    group                (index) int64 14kB 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 1 0 2
Attributes: (12/22)
    Conventions:                       CF-1.6
    creator_name:                      Jet Propulsion Laboratory/California I...
    creator_url:                       aviris.jpl.nasa.gov
    date_created:                      2025-04-23T20:50:10Z
    grid_cell:                         3217
    identifier_product_doi_authority:  https://doi.org
    ...                                ...
    sensor:                            Airborne Visible / Infrared Imaging Sp...
    software_build_version:            2
    summary:                           Mosaic of AVIRIS-NG L2A Reflectance da...
    time_coverage_end:                 2024-11-26T23:59:59Z
    time_coverage_start:               2023-10-22T00:00:00Z
    title:                             AVIRIS-NG L3 Mosaiced Surface Reflecta...

In [17]:
dsp.xvec.to_geodataframe()

MemoryError: Unable to allocate 10.4 TiB for an array with shape (425, 2000, 2000, 1, 1688) and data type float32